<a href="https://colab.research.google.com/github/lordtrump/HF/blob/main/KoboldCpp(CUN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title <b> [1]  If you play on mobile, tap this to open music player and play the white noise to keep tab running in the background. or google will kill your api
%%html
<b>Press play on the music player to keep the tab alive, then start block below (Uses only 13MB of data)</b><br/>
<audio src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" controls>

In [ ]:


# @title <b>[2] Select model and etc</b> {"display-mode":"form"}

import os
import time
if not os.path.isfile("/opt/bin/nvidia-smi"):
  raise RuntimeError("⚠️Colab did not give you a GPU because you used it to often recently, this can take a few hours before they let you back in. Try again later or subscribe to Colab Pro for immediate access. (or change email if you are really desperate)⚠️")
import time
from google.colab import runtime
import json
import re
from IPython.display import clear_output

# @markdown ### DO NOT CLOSE THIS TAB WHILE USING, GOOGLE ANTI AFK WILL KILL API

Model = "M7-Evil (7B)"  # @param ["M7-Evil (7B)","Daybreak-Kunoichi (7B)","Daybreak-TheSpice (8B)","Gemma2-Daybreak (9B)","Magnum (12B)","Sunfall-NemoMix (12B)","Nemo-Sunfall (12B)"]{allow-input: true}
Context = "4096" # @param ["65536","32768","24576","16384","12288","8192","6144","4096", "3072", "2048"]

Instruct_Preset = "alpaca" # @param ["None","alpaca", "vicuna", "llama-3", "command-r", "chatml", "mistral", "gemma2", "metharme"]
# @markdown model will works best if using the same instruct it was trained for. certain model (e.g. all llama3 models) will become idiot if the instruct is mismatched. Please refer to the table above.


#@markdown ---
#@markdown ### Advanced setting
Layers = 99 # @param {type:"number"}
# @markdown Reduce layer will make generation slower, but it also save VRAM. So you can have more context size.
KvCache = "0" # @param ["0", "1", "2"]
# @markdown Kvcache will reduce a bit of respose quality to save VRAM. But it will reprocess prompt every time new message is sent.

blayer = 99


#@markdown ---
force_redownload_kobold = True # @param {type:"boolean"}

provider = "Cloudflare" # @param ["Cloudflare", "Localtunnel"]
# @markdown ##### If stuck at "Checking if the server is up..." while using cloudflare, change provider to localtunnel may help
# @markdown ##### Check force redownload if you try everything but it still not working.
modellink = ''

# declare context template

premade_instruct = {
    "alpaca": {
        "system_start": "\n### Input: ",
        "system_end": "",
        "user_start": "\n### Instruction: ",
        "user_end": "",
        "assistant_start": "\n### Response: ",
        "assistant_end": "",
    },
    "vicuna": {
        "system_start": "\nSYSTEM: ",
        "system_end": "",
        "user_start": "\nUSER: ",
        "user_end": "",
        "assistant_start": "\nASSISTANT: ",
        "assistant_end": "",
    },
    "llama-3": {
        "system_start": "<|start_header_id|>system<|end_header_id|>\n\n",
        "system_end": "<|eot_id|>",
        "user_start": "<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n",
        "user_end": "<|eot_id|>",
        "assistant_start": "<|start_header_id|>assistant<|end_header_id|>\n\n",
        "assistant_end": "<|eot_id|>",
    },
    "chatml": {
        "system_start": "<|im_start|>system",
        "system_end": "<|im_end|>",
        "user_start": "<|im_start|>user",
        "user_end": "<|im_end|>",
        "assistant_start": "<|im_start|>assistant",
        "assistant_end": "<|im_end|>",
    },
    "command-r": {
        "system_start": "<|START_OF_TURN_TOKEN|><|SYSTEM_TOKEN|>",
        "system_end": "<|END_OF_TURN_TOKEN|>",
        "user_start": "<|START_OF_TURN_TOKEN|><|USER_TOKEN|>",
        "user_end": "<|END_OF_TURN_TOKEN|>",
        "assistant_start": "<|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>",
        "assistant_end": "<|END_OF_TURN_TOKEN|>",
    },
    "mistral":  {
      "system_start": "",
      "system_end": "",
      "user_start": "[INST] ",
      "user_end": "",
      "assistant_start": " [/INST]",
      "assistant_end": "</s> "
    },
    "gemma2":{
      "system_start": "<start_of_turn>system\n",
      "system_end": "<end_of_turn>\n",
      "user_start": "<start_of_turn>user\n",
      "user_end": "<end_of_turn>\n",
      "assistant_start": "<start_of_turn>model\n",
      "assistant_end": "<end_of_turn>\n"
    },
    "metharme": {
      "system_start": "<|system|>",
      "system_end": "",
      "user_start": "<|user|>",
      "user_end": "",
      "assistant_start": "<|model>",
      "assistant_end": ""
    }
}

if "https" in Model:
  modellink = Model
else:
  match Model:
    case "M7-Evil (7B)" :
        modellink = "https://huggingface.co/InferenceIllusionist/M7-Evil-7b-GGUF/resolve/main/m7-evil-7b-Q6_K.gguf"
    case "Daybreak-Kunoichi (7B)" :
        modellink = "https://huggingface.co/crestf411/daybreak-kunoichi-2dpo-7b-gguf/resolve/main/daybreak-kunoichi-2dpo-7b-q8_0.gguf?download=true"
    case "Daybreak-TheSpice (8B)" :
        modellink = "https://huggingface.co/Vdr1/L3-daybreak-TheSpice-8b-v0.1.3-GGUF/resolve/main/L3-daybreak-TheSpice-8b-v0.1.3-Q4_K_M.gguf?download=true"
    case "Gemma2-Daybreak (9B)" :
        modellink = "https://huggingface.co/Vdr1/gemma2-9B-daybreak-v0.5-GGUF-Imatrix-IQ/resolve/main/gemma2-9B-daybreak-v0.5-Q4_K_S-imat.gguf?download=true"
    case "Magnum (12B)" :
        modellink = "https://huggingface.co/NikolayKozloff/magnum-12b-v2.5-kto-Q5_K_M-GGUF/resolve/main/magnum-12b-v2.5-kto-q5_k_m.gguf"
    case "Sunfall-NemoMix (12B)" :
        modellink = "https://huggingface.co/Vdr1/Sunfall-NemoMix-Unleashed-12B-v0.6.1-GGUF-IQ/resolve/main/Sunfall-NemoMix-Unleashed-12B-v0.6.1-Q4_K_M.gguf?download=true"
    case "Nemo-Sunfall (12B)" :
        modellink = "https://huggingface.co/Vdr1/nemo-sunfall-v0.6.1-GGUF-IQ/resolve/main/nemo-sunfall-v0.6.1-Q4_K_M.gguf?download=true"


!echo Downloading KoboldCpp, please wait...
!wget -O dlfile.tmp https://kcpplinux.concedo.workers.dev && mv dlfile.tmp koboldcpp_linux
!test -f koboldcpp_linux && echo Download Successful || echo Download Failed
!chmod +x ./koboldcpp_linux
!apt update
!apt install aria2 -y
clear_output()

modelname = modellink.split('/')[-1].split('.')[0]
matched = re.search(r'/([^/]+)-GGUF/', modellink)
remodelname = ("koboldcpp/model")
if matched:
    remodelname = f"koboldcpp/{matched.group(1)}"
isModelExist = os.path.isfile('model.txt')
oldmodel = ""
if(isModelExist):
  f = open("model.txt", "r")
  oldmodel = f.read()
  print(f'oldmodel = {oldmodel}')

if(oldmodel != modellink or isModelExist == False):
  print('Download: '+ modellink)
  !aria2c -c -x 16 -s 16 -k 1M -o model.gguf --download-result=default --allow-overwrite=true --file-allocation=none --console-log-level=error $modellink
  # !aria2c -x 16 -s 16 -k 1M --allow-overwrite="true" --summary-interval=5 $modellink -o model.gguf 2>&1 | grep -Ev 'Redirecting'
  with open('model.txt', 'w') as f:
      f.write(modellink)
print(modelname)
clear_output()

#create instruct file
with open("instruct.json", "w") as f:
    f.write(json.dumps(premade_instruct[Instruct_Preset], separators=(",", ":")))


if provider == "Localtunnel":
  !npm install -g localtunnel
  print('\n')
  !echo > nohup.out
  !nohup lt --port 5001 &
  print("Checking if the server is up...\n")
  while True:
      time.sleep(1)
      with open('nohup.out', 'r') as f:
        if 'your url is' in f.read():
            print('=============================================================================')
            print('please verify ip of colab in the loca.lt link before using it as kobold url')
            print('colab ip: ', end='')
            !curl ipecho.net/plain
            !cat nohup.out
            print('===========================================================================')
            print("--------------------------\nServer up!")
            break
  print("--------------------------\n")

tunneling = ""
if provider == "Cloudflare":
  tunneling = "--remotetunnel"
!./koboldcpp_linux model.gguf --usecublas 0 mmq --multiuser --gpulayers $Layers --contextsize $Context --flashattention --hordemodelname $remodelname --quiet --remotetunnel --chatcompletionsadapter instruct.json